In [1]:
import os
from pathlib import Path
# Change cwd to the project root (parent of 'notebooks/')
os.chdir(Path.cwd().parent)
Path.cwd()

PosixPath('/Users/jbrandt/code/birddog')

In [2]:
from birddog.core import Archive

2025-04-22 22:00:45,587 [INFO] Using Google Cloud translation API (credentials file:/Users/jbrandt/code/birddog/google-cloud-translate-key.json)
2025-04-22 22:00:45,621 [INFO] Found credentials in environment variables.
2025-04-22 22:00:45,681 [INFO] Using AWS S3 bucket birddog-data for storage.


In [3]:
page = Archive("DAZHO")
page = page.lookup('1').lookup('74')

2025-04-22 22:00:45,903 [INFO] HistoryLRU.lookup(ДАЖО/Д): cache miss
2025-04-22 22:00:46,098 [INFO] fetch_url: 1 requests in last 60s → 0.02 req/s
2025-04-22 22:00:46,774 [INFO] Retrieved from cache: DAZHO-D[2025,01,19,12:47]: page_cache/DAZHO-D/2025,01,19,12:47.json
2025-04-22 22:00:46,775 [INFO] HistoryLRU.lookup(ДАЖО/1): cache miss
2025-04-22 22:00:47,066 [INFO] Retrieved from cache: DAZHO-D/1[2025,04,21,21:06]: page_cache/DAZHO-D/1/2025,04,21,21:06.json
2025-04-22 22:00:47,067 [INFO] HistoryLRU.lookup(ДАЖО/1/74): cache miss
2025-04-22 22:00:47,271 [INFO] Loading page: DAZHO-D/1/74 from https://uk.wikisource.org/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%96%D0%9E/1/74
2025-04-22 22:00:47,516 [INFO] saving page_cache/DAZHO-D/1/74/2025,04,22,20:17.json: 280741


In [4]:
from birddog.ai import AWSBedrockClient, classify_table_columns, _form_table_column_classifier_prompt

In [5]:
client = AWSBedrockClient()

In [6]:
#client.write("Why is the sky blue?")

In [7]:
classes = {
    "DATE": "A column indicating a single date or a date range",
    "DESCRIPTION": "A column containing a textual description of the item",
    "ID": "A unique row identifier, number, or code"
}

In [8]:
headers = [col['uk'] for col in page.header]
print(headers)
max_rows = 5
rows = [ [ item['text']['uk'] for item in row ] for row in page.children[:max_rows] ]
print(rows)

['№', 'Назва', 'Повіт', 'Роки', 'Аркушів', 'Посилання на FamilySearch']
[['1', 'Метричні книги Кашогродського уніатського деканату Ковельського і Луцького повітів', 'Ковельський, Луцький', '1798, 1802–1803, 1813', '319', ''], ['2', 'Сповідні розписи Ратинського уніатського деканату Ковельського повіту', 'Ковельський', '1804', '189', ''], ['3', 'Метричні книги уніатських церков Ковельського повіту', 'Ковельський', '1804', '138', ''], ['4', 'Метричні книги Кашогродського уніатського деканату Ковельського і Луцького повітів', 'Ковельський, Луцький', '1820', '127', ''], ['5', 'Метричні книги уніатських церков Ковельського та Луцького повітів', 'Ковельський, Луцький', '1821, 1822', '148', '']]


In [9]:
full_prompt = _form_table_column_classifier_prompt(headers, classes, rows, max_rows=len(rows))
print(full_prompt)

You are given a list of table column headers in Ukranian and their indices:
0: №
1: Назва
2: Повіт
3: Роки
4: Аркушів
5: Посилання на FamilySearch

Classify each header into one of the following types based on the descriptions below:
- DATE: A column indicating a single date or a date range
- DESCRIPTION: A column containing a textual description of the item
- ID: A unique row identifier, number, or code

Here are some sample rows of the table:
Row 1: ['1', 'Метричні книги Кашогродського уніатського деканату Ковельського і Луцького повітів', 'Ковельський, Луцький', '1798, 1802–1803, 1813', '319', '']
Row 2: ['2', 'Сповідні розписи Ратинського уніатського деканату Ковельського повіту', 'Ковельський', '1804', '189', '']
Row 3: ['3', 'Метричні книги уніатських церков Ковельського повіту', 'Ковельський', '1804', '138', '']
Row 4: ['4', 'Метричні книги Кашогродського уніатського деканату Ковельського і Луцького повітів', 'Ковельський, Луцький', '1820', '127', '']
Row 5: ['5', 'Метричні книг

In [10]:
client.write(full_prompt)

2025-04-22 22:00:48,679 [INFO] Found credentials in environment variables.


'["ID","DESCRIPTION","OTHER","DATE","OTHER","OTHER"]'

In [11]:
import json
json.loads(client.write(full_prompt))

['ID', 'DESCRIPTION', 'OTHER', 'DATE', 'OTHER', 'OTHER']

In [12]:
classify_table_columns(page)

2025-04-22 22:00:52,417 [INFO] Creating AWS Bedrock client
2025-04-22 22:00:52,430 [INFO] Found credentials in environment variables.
2025-04-22 22:00:53,632 [INFO] table_column_classifier result: ["ID","DESCRIPTION","OTHER","DATE","OTHER","OTHER"]


{'mapping': ['ID', 'DESCRIPTION', 'OTHER', 'DATE', 'OTHER', 'OTHER'],
 'success': True}